## CHSH game

with Qiskit

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_ibm_runtime import Sampler, QiskitRuntimeService
from qiskit_ibm_runtime.ibm_backend import IBMBackend
from qiskit_ibm_runtime.runtime_job_v2 import RuntimeJobV2
from qiskit_aer import Aer
from numpy import pi as π

In [ ]:
def CHSH_circuit(x: int, y: int, alpha0: float = 0, alpha1: float = π/2, 
                 beta0: float = π/4, beta1: float = -π/4) -> QuantumCircuit:
    """Implementation of the strategy in form of a circuit.

    Args:
        x  (int): Alice's classical bit
        y  (int): Bob's classical bit
        alpha0 (float): measure angle used by Alice when she receives 0
        alpha1 (float): measure angle used by Alice when she receives 1
        beta0 (float): measure angle used by Bob when she receives 0
        beta1 (float): measure angle used by Bob when she receives 1

    Returns:
        QuantumCircuit: Constructed quantum circuit
    """
    # instantiate a circuit
    qr = QuantumRegister(2, "qr")
    cr = ClassicalRegister(2, "cr")

    qc = QuantumCircuit(qr, cr)

    # create a Bell pair
    qc.h(0)
    qc.cx(0, 1)

    # apply rotations based on obtained bits
    if x == 0:
        qc.ry(alpha0, 0)
    else:
        qc.ry(alpha1, 0)

    if y == 0:
        qc.ry(beta0, 1)
    else:
        qc.ry(beta1, 1)

    qc.measure(range(2), range(2))

    return qc   


In [64]:
def submit_job(backend: IBMBackend, shots: int = 4096, alpha0: float = 0, 
               alpha1: float = π/2, beta0: float = π/4, beta1: float = -π/4, 
               sim: bool = False) -> tuple:
    """Submit the job for computation.

    Args:
        backend (IBMBackend): Chosen backed to run the algorithm on.
        shots (int, optional): Number of shots, to gain statistics. 
            Defaults to 4096.
        alpha0 (float, optional): measure angle used by Alice when she 
            receives 0. Defaults to 0.
        alpha1 (float, optional): measure angle used by Alice when she 
            receives 1. Defaults to π/2.
        beta0 (float, optional): measure angle used by Bob when he receives 0. 
            Defaults to π/4.
        beta1 (float, optional): measure angle used by Bob when he receives 1. 
            Defaults to -π/4.
        sim (bool, optional): Whether the chosen backend is a simulator or an 
            actual quantum hardware. Defaults to False.

    Returns:
        tuple: Job and the quantum circuit (RuntimeJobV2, list[QuantumCircuit])
    """

    qcs = [CHSH_circuit(0, 0, alpha0, alpha1, beta0, beta1), 
           CHSH_circuit(0, 1, alpha0, alpha1, beta0, beta1), 
           CHSH_circuit(1, 0, alpha0, alpha1, beta0, beta1), 
           CHSH_circuit(1, 1, alpha0, alpha1, beta0, beta1)]

    comp_qcs = [transpile(qci, backend) for qci in qcs]
    if sim:
        sampler = backend
    else:
        sampler = Sampler(backend)
    job = sampler.run(comp_qcs, shots=shots)

    return job, qcs



def winning_probability(job: RuntimeJobV2, qcs: list[QuantumCircuit], 
                        shots: int = 4096, sim: bool = False) -> float:
    """Compute probability of Alice and Bob winning the game.

    Wrapper to conveniently call pending jobs.

    Args:
        job (RuntimeJobV2): Submitted job.
        qcs (list[QuantumCircuit]): List of quantum circuits 
            associated with a job.
        shots (int, optional): Number of shots that were used. 
            Defaults to 4096.
        sim (bool, optional): Whether the chosen backend is a simulator or an 
            actual quantum hardware. Defaults to False.

    Returns:
        float: Total probability of Alice and Bob winning.
    """

    # For the first three circuits, the winning condition is that Alice's and 
    # Bob's outputs are equal
    total = 0
    for idx, qc in enumerate(qcs[:3]):
        if sim:
            counts = job.result().get_counts(qc)
        else:
            counts = job.result()[idx].data.cr.get_counts()

        if "00" in counts:
            total += counts["00"]
        if "11" in counts:
            total += counts["11"]

    # For the fourth circuit, the outputs must be different for them to win
    if sim:
        counts = job.result().get_counts(qcs[-1])
    else:
        counts = job.result()[-1].data.cr.get_counts()
    if "01" in counts:
        total += counts["01"]
    if "10" in counts:
        total += counts["10"]

    return total / (4 * shots)

##### Quantum simulator

In [32]:
backend = Aer.get_backend("qasm_simulator")

sim_job, sim_qcs = submit_job(backend, sim=True)
sim_prob = winning_probability(sim_job, sim_qcs, sim=True)

print(f"Winning probability is equal to {sim_prob:.5f}.")

Winning probability is equal to 0.85742.


##### Quantum hardware

In [58]:
# get backend
service = QiskitRuntimeService()
backend = service.least_busy()
print(f"Using {backend.name}")

qiskit_runtime_service.__init__:WARNING:2025-11-03 20:49:34,316: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: eloza. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-03 20:49:34,929: Using instance: eloza, plan: open


Using ibm_fez


In [ ]:
# run the experiment
qh_job, qh_qcs = submit_job(backend)
# qh_job = service.job("d44fpnvlcjfs73atmnkg")
qh_prob = winning_probability(qh_job, qh_qcs)

print(f"Winning probability is equal to {qh_prob:.5f}.")

Winning probability is equal to 0.83722.
